# Import

## Library

In [64]:
# Import Libraries
import os
import pandas as pd
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# Visuzliation Setting
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib import rc
from matplotlib import colors
import seaborn as sns

## Data

In [65]:
root = os.path.join(os.getcwd(), 'DATA')

In [66]:
df_raw = pd.read_excel(os.path.join(root, 'train.xlsx'))
df_train = df_raw.copy()

In [101]:
df_weather1 = pd.read_csv(os.path.join(root, 'GTS_SYNOP_TIM_20151228-20161227.csv'), encoding='euc-kr')
df_weather2 = pd.read_csv(os.path.join(root, 'GTS_SYNOP_TIM_20161228-20171227.csv'), encoding='euc-kr')
df_weather3 = pd.read_csv(os.path.join(root, 'GTS_SYNOP_TIM_20171228-20181227.csv'), encoding='euc-kr')
df_weather4 = pd.read_csv(os.path.join(root, 'GTS_SYNOP_TIM_20181228-20191227.csv'), encoding='euc-kr')
df_weather_code = pd.read_excel(os.path.join(root, '강수량번호국가매칭.xlsx'), header=None)

In [ ]:
df_salinity = pd.read_csv(os.path.join(root, ''))

# Preprocess

In [67]:
df_train['year'] = df_train['REG_DATE'].dt.year
df_train['month'] = df_train['REG_DATE'].dt.month
df_train['day'] = df_train['REG_DATE'].dt.day

In [68]:
import_type_list = set()
for tmp in df_train.P_IMPORT_TYPE.unique():
    for a in tmp.split(','):
        import_type_list.add(a)

In [69]:
for name in import_type_list:
    df_train[name] = 0
    df_train.loc[df_train['P_IMPORT_TYPE'].str.contains(name, regex=False), name] = 1

In [70]:
df_train['VALUE_COUNT'] = 0;
value_dict = {}
for name, value in zip(df_train['P_NAME'].value_counts().index,df_train['P_NAME'].value_counts()):
    value_dict[name] = value

def value(col):
    return value_dict[col]

df_train['VALUE_COUNT'] = df_train['P_NAME'].apply(value)

## add weather data

In [123]:
df_train['wind'] = np.NaN
df_train['rain'] = np.NaN
df_train['temp'] = np.NaN

**전처리해햘 부분**
- "지점/지점명"을 기본데이터의 CTYR_1의 나라명과 맞춰줘야함
    - df_weather_code을 사용하여 변경 필요
    - df_wather# 에 CTTRY_1 컬럼을 추가하여 사용
- 하루에 한 지점에서 측정한 데이터가 여러개임
    - 하루에 한 데이터가 나오도록 전처리 필요
- 강수량의 경우 NaN 값이 너무 많음
    - 위 전처리들을 한 후에도 많다면 사용불가
    - 위 전처리 후에는 적다면 전날/다음날 데이터 기반으로 채워넣기 진행

In [ ]:
df_train['CTRY_1']

In [115]:
df_weather1[df_weather1['일시'].astype('str').str.contains('2015-12-28')]

,지점,지점명,일시,풍속,강수량,기온
0,1001,얀마이엔,2015-12-28 03:00,11.0,NaN,2.3
1,1001,얀마이엔,2015-12-28 06:00,13.0,0.0,2.6
2,1001,얀마이엔,2015-12-28 09:00,11.0,NaN,3.3
3,1001,얀마이엔,2015-12-28 12:00,13.0,0.1,3.4
4,1001,얀마이엔,2015-12-28 15:00,15.0,NaN,3.7
...,...,...,...,...,...,...
388384,85874,발마세다,2015-12-28 12:00,22.0,NaN,15.1
388385,85874,발마세다,2015-12-28 18:00,22.0,0.0,16.4
389093,85934,푼타아레나스,2015-12-28 06:00,3.0,NaN,3.7
389094,85934,푼타아레나스,2015-12-28 12:00,4.0,NaN,8.8


In [109]:
df_raw['CTRY_1'].unique()

array(['아르헨티나', '바레인', '칠레', '중국', '프랑스', '인도네시아', '일본', '대한민국', '모로코',
       '멕시코', '말레이시아', '노르웨이', '페루', '러시아', '시에라리온', '세네갈', '태국', '튀니지',
       '대만', '미국', '베트남', '남아프리카 공화국', '기니', '인도', '필리핀', '포르투갈',
       '사우디아라비아', '뉴질랜드', '파키스탄', '아랍에미리트', '캐나다', '에콰도르', '아일랜드', '터키',
       '스페인', '아이슬란드', '모리타니', '호주', '바누아투', '영국', '소말리아', '불가리아', '피지',
       '우크라이나', '리비아', '세이셸', '이탈리아', '키리바시', '기니비사우', '브라질', '스리랑카',
       '우루과이', '세인트빈센트 그레나딘', '에스토니아', '이란', '미얀마', '파푸아뉴기니', '가나',
       '방글라데시', '네덜란드', '파나마', '감비아', '크로아티아', '앙골라', '베네수엘라', '싱가포르',
       '포클랜드 제도', '모잠비크', '이집트', '수리남', '루마니아', '몰타', '라트비아', '나미비아',
       '오만', '그리스', '그린란드', '알제리', '니카라과', '쿡 제도', '팔라우', '쿠바', '라이베리아',
       '마다가스카르', '미크로네시아 연방', '사이프러스', '콜롬비아'], dtype=object)

## add salinity

## onehot encoding

In [71]:
one_hot = ['CTRY_1', 'CTRY_2', 'P_PURPOSE', 'CATEGORY_1', 'CATEGORY_2', 'P_NAME']

In [72]:
df_train = pd.get_dummies(df_train, columns=one_hot)

## drop columns

In [ ]:
drop = ['REG_DATE', 'P_TYPE', 'P_IMPORT_TYPE']

In [73]:
df_train.drop(columns = drop, inplace=True)

# Train

In [74]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.ensemble import AdaBoostRegressor

In [75]:
target = df_train['P_PRICE']
df_train.drop(columns = 'P_PRICE', inplace=True)

x_train, x_test, y_train, y_test = train_test_split(df_train, target, random_state = 0)

In [ ]:
model = RandomForestRegressor(n_estimators=500,random_state=0)
model.fit(x_train,y_train)
cross = cross_val_score(model,x_test,y_test,cv = 5)

print('cross_tree',cross.mean())

In [ ]:
from sklearn.svm import SVR
model = SVR()
model.fit(x_train,y_train)
cross_svr = cross_val_score(model,x_test,y_test,cv = 5)
print('cross_svr',cross_svr.mean())

In [ ]:
from xgboost import XGBRegressor
model = XGBRegressor(n_estimators=500,learning_rate=0.05,max_depth=3)
model.fit(x_train,y_train)
cross_xgboost = cross_val_score(model,x_test,y_test,cv = 5)

print('cross_xgboost : ',cross_xgboost.mean())

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(x_train,y_train)

cross_linear = cross_val_score(model,x_test,y_test,cv = 5)
print(cross_linear)
print('cross_linear : ',cross_linear.mean())

In [ ]:
from sklearn.neural_network import MLPRegressor

model = MLPRegressor().fit(x_train,y_train)
cross_MLP = cross_val_score(model,x_test,y_test,cv = 5)

print('croos_MLP : ',cross_MLP.mean())